In [1]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset
from sklearn.model_selection import train_test_split
from surprise import SVD, SVDpp, NMF
from cmfrec import CMF
from utility_surprise import run_surprise
from utility_cmfrec import run_cmfrec

Load data

In [2]:
ratings = pd.read_csv("../../preprocessed_data/ratings.csv")

Prepare data

In [3]:
rec_train, rec_test = train_test_split(ratings,test_size= 0.2, random_state= 42)
users_train,games_train = rec_train["user_id"].unique(), rec_train["app_id"].unique()
rec_test =rec_test.loc[rec_test["user_id"].isin(users_train) & rec_test["app_id"].isin(games_train)]
testset = list(zip(rec_test.user_id.values, rec_test.app_id.values, rec_test.explicit_rating.values))
print(rec_train.shape)
print(rec_test.shape)



(3483739, 4)
(870400, 4)


In [4]:
min_rating = rec_train.explicit_rating.min()
max_rating = rec_train.explicit_rating.max()
reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(rec_train[['user_id', 'app_id', 'explicit_rating']], reader)
trainset = data.build_full_trainset() 
testset = list(zip(rec_test.user_id.values, rec_test.app_id.values, rec_test.explicit_rating.values))

In [5]:
svd = SVD(random_state = 42, n_factors = 50, n_epochs = 50 ,verbose = True, lr_all= 0.05, reg_all=0.15)
train,test= run_surprise(svd, trainset, testset)

Training the model...
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processi

In [6]:
svdpp = SVDpp(random_state = 42, n_factors = 50, n_epochs = 50 ,verbose = True, lr_all= 0.05, reg_all= 0.15)
train,test= run_surprise(svdpp,trainset, testset)

Training the model...
 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
 processing epoch 20
 processing epoch 21
 processing epoch 22
 processing epoch 23
 processing epoch 24
 processing epoch 25
 processing epoch 26
 processing epoch 27
 processing epoch 28
 processing epoch 29
 processing epoch 30
 processing epoch 31
 processing epoch 32
 processing epoch 33
 processing epoch 34
 processing epoch 35
 processing epoch 36
 processing epoch 37
 processing epoch 38
 processing epoch 39
 processing epoch 40
 processing epoch 41
 processing epoch 42
 processing epoch 43
 processing epoch 44
 processing epoch 45
 processing epoch 46
 

In [7]:
nmf = NMF(random_state = 42, n_factors = 40, n_epochs = 50 ,verbose = True)
train,test= run_surprise(nmf, trainset, testset)

Training the model...
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processi

In [9]:
from datetime import datetime
import numpy as np
import random
from collections import defaultdict
from math import log2

my_seed = 15
random.seed(my_seed)
np.random.seed(my_seed)

def get_ratings(predictions):
    actual = np.array([pred.r_ui for pred in predictions])
    pred = np.array([pred.est for pred in predictions])
    return actual, pred
def get_errors(predictions):
    actual, pred = get_ratings(predictions)
    rmse = np.sqrt(np.mean((pred - actual)**2))
    mape = np.mean(np.abs(actual-pred)/actual) * 100
    return rmse, mape
def evaluation(predictions, k=5, threshold=3.5):
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
    precisions = defaultdict()
    recalls = defaultdict()
    ndcgs = defaultdict()
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        dcg = sum(true_r / log2(i+2) if true_r >= threshold else 0 for i,(_, true_r) in enumerate(user_ratings[:k]))
        sorted_true = sorted(user_ratings, key = lambda x: x[1], reverse = True)
        idcg = sum(true_r / log2(i+2) if true_r >= threshold else 0 for i,(_, true_r) in enumerate(sorted_true[:k]))
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:10])
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:10]
        )
        ndcgs[uid] = dcg/idcg if idcg!= 0 else 0
        precisions[uid]= n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid]= n_rel_and_rec_k / n_rel if n_rel != 0 else 0
    precision = sum(prec for prec in precisions.values()) / len(precisions)
    recall = sum(rec for rec in recalls.values()) / len(recalls)
    ndcg = sum(nd for nd in ndcgs.values()) / len(ndcgs)
    f1_score = sum((2*precisions[i]*recalls[i]/(precisions[i]+ recalls[i])) if precisions[i] +recalls[i] != 0 else 0 for i in (precisions.keys()))/len(precisions)

    return precision, recall, f1_score , ndcg
def run_surprise(algo, trainset, testset, verbose=True): 
    start = datetime.now()
    train = dict()
    test = dict()
    st = datetime.now()
    print('Training the model...')
    algo.fit(trainset)
    print('Done. time taken : {} \n'.format(datetime.now()-st))
    st = datetime.now()
    print('Evaluating the model with train data..')
    train_preds = algo.test(trainset.build_testset())
    precision, recall,f1, ndcg = evaluation(train_preds)
    train_rmse, train_mape = get_errors(train_preds)
    print('time taken : {}'.format(datetime.now()-st))
    if verbose:
        print('-'*15)
        print('Train Data')
        print('-'*15)
        print("RMSE : {}\n\nMAPE : {}\n".format(train_rmse, train_mape))
        print()
    if verbose:
        print('adding train results in the dictionary..')
    train['rmse'] = train_rmse
    train['mape'] = train_mape
    train['recall'] = recall
    train['precision'] = precision
    train['f1'] = f1
    st = datetime.now()
    print('\nEvaluating for test data...')
    test_preds = algo.test(testset)
    test_rmse, test_mape = get_errors(test_preds)
    precision, recall,f1,ndcg = evaluation(test_preds)
   
    print('time taken : {}'.format(datetime.now()-st))
    if verbose:
        print('-'*15)
        print('Test Data')
        print('-'*15)
        print("RMSE : {}\n\nMAPE : {}\n".format(test_rmse, test_mape))
        print("Precision@10 : {}\n\nRecall@10 : {}\n\nF1@10 : {}\n\nNDCG@5: {}".format(precision, recall, f1, ndcg))
    if verbose:
        print('storing the test results in test dictionary...')
    test['rmse'] = test_rmse
    test['mape'] = test_mape
    print('\n'+'-'*45)
    print('Total time taken to run this algorithm :', datetime.now() - start)
    return train,test

In [11]:
als = CMF(k= 50 , method = 'als', lambda_ = 35)
train, test = run_cmfrec(als , rec_train, rec_test)

Training the model...
Done. Time taken: 0:00:34.764835 

Evaluating the model with train data...
Time taken: 0:00:22.865107
---------------
Train Data
---------------
RMSE: 0.8397834611442527

MAE: 28.767264846897323


Adding train results to the dictionary...

Evaluating the model with test data...
Time taken: 0:00:18.009073
---------------
Test Data
---------------
RMSE: 0.8397834611442527

MAE: 28.767264846897323

Precision@10: 0.9503289434957759

Recall@10: 0.28261004622987496

F1@10: 0.4222205324876224

NDCG@5: 0.9511299262006849
Storing the test results in the test dictionary...

---------------------------------------------
Total time taken to run this algorithm: 0:01:15.806487
